Importing Necessary Libraries

In [91]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

Reading Cropped Face Images From Folder, Converting Them To Gray Scale & Resizing Them Into 64*64

In [92]:
main_folder_path = "/kaggle/input/cropped-faces/cropped_faces"
X = []
y = []

for folder in os.listdir(main_folder_path):
    category_path = os.path.join(main_folder_path, folder)
    for image_name in os.listdir(category_path):
        image_path = os.path.join(category_path, image_name)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (64, 64))
#         print(img.shape)
        
        X.append(img)
        y.append(folder)

Converting X & y into Numpy Array

In [93]:
# Convert the list of strings to a NumPy array of strings
s = set(y)
print(len(s))
y = np.array(y, dtype='str')
X = np.array(X)
print(X.shape)
print(y.shape)

49
(734, 64, 64)
(734,)


One Hot Encoding y 

In [94]:
from tensorflow.keras.utils import to_categorical
label_to_int = {label: idx for idx, label in enumerate(np.unique(y))}
y_int = np.array([label_to_int[label] for label in y])

# Assuming your labels are in the form of integers (e.g., class indices)
# You should replace this with your actual labels
y_encoded = to_categorical(y_int, num_classes=49)
# Now, you can use y_train_encoded as your target data for training

In [95]:
y_int.shape

(734,)

In [96]:
y_encoded.shape

(734, 49)

Adding One More Dimension to X and Train Test Split

In [97]:
np.random.seed(1)
tf.random.set_seed(1)
X = np.expand_dims(X, axis=3)
print('Shape of X : {}, Shape of y : {}'.format(X.shape, y.shape))
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=1)

Shape of X : (734, 64, 64, 1), Shape of y : (734,)


Building Our DNN

In [98]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(64, 64, 1)),  # Flatten the 256x256x1 input
    keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(49, activation='softmax')
])

In [99]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 4096)              0         
                                                                 
 dense_28 (Dense)            (None, 512)               2097664   
                                                                 
 dropout_21 (Dropout)        (None, 512)               0         
                                                                 
 dense_29 (Dense)            (None, 512)               262656    
                                                                 
 dropout_22 (Dropout)        (None, 512)               0         
                                                                 
 dense_30 (Dense)            (None, 256)               131328    
                                                                 
 dropout_23 (Dropout)        (None, 256)              

Training

In [100]:
datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, 
                            horizontal_flip=True)
datagen.fit(X_train)
batch_size=32
model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch =
                          len(X_train) // batch_size, epochs=200, validation_data=(X_test, y_test))

Epoch 1/200


/tmp/ipykernel_28/3313357855.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch =


16/16 [==============================] - 2s 20ms/step - loss: 20.6699 - accuracy: 0.0977 - val_loss: 588.9064 - val_accuracy: 0.0181
Epoch 2/200
16/16 [==============================] - 0s 8ms/step - loss: 17.7218 - accuracy: 0.2017 - val_loss: 605.3130 - val_accuracy: 0.0000e+00
Epoch 3/200
16/16 [==============================] - 0s 8ms/step - loss: 15.0383 - accuracy: 0.3389 - val_loss: 641.6004 - val_accuracy: 0.0226
Epoch 4/200
16/16 [==============================] - 0s 8ms/step - loss: 12.9660 - accuracy: 0.3909 - val_loss: 555.7413 - val_accuracy: 0.0317
Epoch 5/200
16/16 [==============================] - 0s 8ms/step - loss: 11.2183 - accuracy: 0.4941 - val_loss: 766.5834 - val_accuracy: 0.0181
Epoch 6/200
16/16 [==============================] - 0s 8ms/step - loss: 9.7208 - accuracy: 0.5696 - val_loss: 692.9811 - val_accuracy: 0.0181
Epoch 7/200
16/16 [==============================] - 0s 8ms/step - loss: 8.5195 - accuracy: 0.6674 - val_loss: 864.6111 - val_accuracy: 0.0362
E

Normalizing Test Data & Predicting

In [101]:
from keras.backend import epsilon
def normalize(X, mean, std):
    return (X - mean) / (std + epsilon())

predicted = model.predict(normalize(X_test, datagen.mean, datagen.std))

7/7 [==============================] - 0s 2ms/step


In [102]:
predicted.shape

(221, 49)

In [103]:
y_test.shape

(221, 49)

Accuracy On Unseen Test Data

In [104]:
import numpy as np

# Assuming 'y_pred' is your predicted softmax distribution and 'y_test' is your true labels

# Convert softmax predictions to class indices (argmax)
y_pred_classes = np.argmax(predicted, axis=1)

# Convert one-hot encoded true labels to class indices (argmax)
y_true_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = np.mean(y_pred_classes == y_true_classes)
print(accuracy)

0.8506787330316742
